<a href="https://colab.research.google.com/github/GAYARRI/recommender/blob/main/Recommender_1_8(by_UNIQUE_query).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers                             ## MODULO 1 : IMPORTAMOS DEPEDENCIAS DE LIBRERIAS
!pip install google-cloud-bigquery
!pip install pandas
import ipywidgets as widgets
from IPython.display import display

In [ ]:
from google.colab import auth                                   ## MODULO 2 : AUTENTICACION DE USUARIO
                                                                # PARAMETROS INICIALES PARA COMENZAR EL PROCESO DE EXTRACCION DE LOS REGISTROS A CONTROLAR PARA LA
from google.cloud import bigquery                               # RECOMENDACIÓN
import pandas as pd
import sqlite3
import unicodedata
import traceback



# Autenticación
auth.authenticate_user()

project_id='projectml-345114'                                                    # el id del proyecto

# Crea un cliente de BigQuery
client = bigquery.Client(project=project_id)                                     # el cliente para la conexion con big query


In [ ]:
def asignatura():
                                                                   ## MODULO 3 : CONTEOS DE LA BASE DE BIGQUERY
                                                                   # ESTA QUEY NOS PERMITIRA SABER LOS REGISTROS QUE TIENE LA BASE ORIGINAL ASI COMO SU CONTENIDO
    dataset_id1 ='maestro'                                         # PARA PODER CONTROLAR LOS CONTENIDOS DE CADA ETIQUETA.
    dataset_id2 ='ai_content'                                      # EN CASO DE NECESITAR MAS CAMPOS DE RESTRICCIONES SE AÑADIRIAN PRIMERO AQUI Y DESPUES A LA QUERY SIGUIENTE
    dataset_id3 ='blinkanalytics'                                  # UNA VEZ YA SEPAMOS EL NÚMERO DE REGISTROS QUE HAN DE SALIR.
    table_id1 = 'maestro_contenido'
    table_id2 = 'maestro_slides'
    table_id3 = 'vm_slides_topics'
    table_id4 = 'slides_difficulty'



    # construimos la consulta

    asignatura = f'''                                               #########################   QUERY PARA ASIGNATURAS UNICAS
            SELECT
                   t1.asignatura_curso_es as ASIGNATURA,
                   count(distinct t1.idslide) as NUM_REGISTROS

            FROM {project_id}.{dataset_id1}.{table_id1} as t1                    # registros originales de la tabla maestro de contenido

                  inner join {project_id}.{dataset_id1}.{table_id2} as t2        # se cruzan en interseccion con la tabla maestro de slides
                  ON t1.idslide = t2.idtransparencia_completo

                  inner join {project_id}.{dataset_id2}.{table_id3} as t3        # se cruzan en interseccion con la tabla de slides generadas
                  ON t1.idslide = t3.idslide

                  inner join {project_id}.{dataset_id3}.{table_id4} as t4        # se cruzan en interseccion con la tabla de dificultad de las actividades
                  ON t1.idslide = t4.id_ejercicio

                  where t1.idcursooriginal=0

                  GROUP BY t1.asignatura_curso_es


    '''
    df_materias=client.query(asignatura).to_dataframe()
    df_materias.replace('Sin definir', ' ', inplace=True)

    return(df_materias)

In [ ]:
df_materias = asignatura()
materias=df_materias
pd.set_option ("display.max_colwidth",None)
lista_materias = materias['ASIGNATURA']
materias=widgets.Dropdown(options=lista_materias,description = 'ASIGN')
pd.set_option ("display.max_colwidth",None)
display(materias)

Dropdown(description='ASIGN', options=(None, 'Otros', 'Español Lengua Extranjera', 'Geografía e Historia', 'In…

In [ ]:
 def curso(asg):
                                                                   ## MODULO 3 : CONTEOS DE LA BASE DE BIGQUERY
                                                                   # ESTA QUEY NOS PERMITIRA SABER LOS REGISTROS QUE TIENE LA BASE ORIGINAL ASI COMO SU CONTENIDO
    dataset_id1 ='maestro'                                         # PARA PODER CONTROLAR LOS CONTENIDOS DE CADA ETIQUETA.
    dataset_id2 ='ai_content'                                      # EN CASO DE NECESITAR MAS CAMPOS DE RESTRICCIONES SE AÑADIRIAN PRIMERO AQUI Y DESPUES A LA QUERY SIGUIENTE
    dataset_id3 ='blinkanalytics'                                  # UNA VEZ YA SEPAMOS EL NÚMERO DE REGISTROS QUE HAN DE SALIR.
    table_id1 = 'maestro_contenido'
    table_id2 = 'maestro_slides'
    table_id3 = 'vm_slides_topics'
    table_id4 = 'slides_difficulty'


    curso = f'''

            SELECT                                                             #########################   QUERY PARA CURSOS UNICOS
                   t1.nivel_curso_es as CURSO

            FROM {project_id}.{dataset_id1}.{table_id1} as t1                    # registros originales de la tabla maestro de contenido

                  inner join {project_id}.{dataset_id1}.{table_id2} as t2        # se cruzan en interseccion con la tabla maestro de slides
                  ON t1.idslide = t2.idtransparencia_completo

                  inner join {project_id}.{dataset_id2}.{table_id3} as t3        # se cruzan en interseccion con la tabla de slides generadas
                  ON t1.idslide = t3.idslide

                  inner join {project_id}.{dataset_id3}.{table_id4} as t4        # se cruzan en interseccion con la tabla de dificultad de las actividades
                  ON t1.idslide = t4.id_ejercicio

            WHERE t1.idcursooriginal=0 AND
                  t1.asignatura_curso_es='{asg}'


            GROUP BY t1.nivel_curso_es



            '''


    df_niveles=client.query(curso).to_dataframe()
    df_niveles.replace('Sin definir', ' ', inplace=True)



    return(df_niveles)


In [ ]:
df_niveles = curso(materias.value)
niveles=df_niveles
pd.set_option ("display.max_colwidth",None)
lista_cursos = niveles['CURSO']
niveles=widgets.Dropdown(options=lista_cursos,description = 'CURSO')
pd.set_option ("display.max_colwidth",None)
display(niveles)

Dropdown(description='CURSO', options=('6º Primaria', '4º ESO', '5º Primaria', '3º ESO', 'Principiante A1', '1…

In [ ]:
def formato(par1,par2):
                                                                   ## MODULO 3 : CONTEOS DE LA BASE DE BIGQUERY
                                                                   # ESTA QUEY NOS PERMITIRA SABER LOS REGISTROS QUE TIENE LA BASE ORIGINAL ASI COMO SU CONTENIDO
    dataset_id1 ='maestro'                                         # PARA PODER CONTROLAR LOS CONTENIDOS DE CADA ETIQUETA.
    dataset_id2 ='ai_content'                                      # EN CASO DE NECESITAR MAS CAMPOS DE RESTRICCIONES SE AÑADIRIAN PRIMERO AQUI Y DESPUES A LA QUERY SIGUIENTE
    dataset_id3 ='blinkanalytics'                                  # UNA VEZ YA SEPAMOS EL NÚMERO DE REGISTROS QUE HAN DE SALIR.
    table_id1 = 'maestro_contenido'
    table_id2 = 'maestro_slides'
    table_id3 = 'vm_slides_topics'
    table_id4 = 'slides_difficulty'
    par1=materias.value
    par2=niveles.value



    formato = f'''

            SELECT                                                                #########################   QUERY PARA TIPOS DE ACTIVIDADES UNICAS
                   t2.tipo_slide as TIPO

            FROM {project_id}.{dataset_id1}.{table_id1} as t1                    # registros originales de la tabla maestro de contenido

                  inner join {project_id}.{dataset_id1}.{table_id2} as t2        # se cruzan en interseccion con la tabla maestro de slides
                  ON t1.idslide = t2.idtransparencia_completo

                  inner join {project_id}.{dataset_id2}.{table_id3} as t3        # se cruzan en interseccion con la tabla de slides generadas
                  ON t1.idslide = t3.idslide

                  inner join {project_id}.{dataset_id3}.{table_id4} as t4        # se cruzan en interseccion con la tabla de dificultad de las actividades
                  ON t1.idslide = t4.id_ejercicio

                  WHERE  t1.idcursooriginal=0 AND t1.nivel_curso_es ='{par2}' AND t1.asignatura_curso_es ='{par1}'

                  GROUP BY t2.tipo_slide


               '''

    df_tipos=client.query(formato).to_dataframe()
    df_tipos.replace('Sin definir', ' ', inplace=True)
    return(df_tipos)


In [ ]:
df_tipos = formato(materias.value,niveles.value)
tipos=df_tipos
pd.set_option ("display.max_colwidth",None)
lista_tipos = tipos['TIPO']
tipos=widgets.Dropdown(options=lista_tipos,description = 'TIPO')
pd.set_option ("display.max_colwidth",None)
display(tipos)

Dropdown(description='TIPO', options=('RESPUESTA MÚLTIPLE', 'RELLENAR ESPACIOS Y SEÑALAR RESPUESTAS', 'VERDADE…

In [ ]:
def dificultad(par1,par2,par3):
                                                                   ## MODULO 3 : CONTEOS DE LA BASE DE BIGQUERY
                                                                   # ESTA QUEY NOS PERMITIRA SABER LOS REGISTROS QUE TIENE LA BASE ORIGINAL ASI COMO SU CONTENIDO
    dataset_id1 ='maestro'                                         # PARA PODER CONTROLAR LOS CONTENIDOS DE CADA ETIQUETA.
    dataset_id2 ='ai_content'                                      # EN CASO DE NECESITAR MAS CAMPOS DE RESTRICCIONES SE AÑADIRIAN PRIMERO AQUI Y DESPUES A LA QUERY SIGUIENTE
    dataset_id3 ='blinkanalytics'                                  # UNA VEZ YA SEPAMOS EL NÚMERO DE REGISTROS QUE HAN DE SALIR.
    table_id1 = 'maestro_contenido'
    table_id2 = 'maestro_slides'
    table_id3 = 'vm_slides_topics'
    table_id4 = 'slides_difficulty'
    par1=materias.value
    par2=niveles.value
    par3=tipos.value


    dificultad = f'''

       SELECT                                                               #########################   QUERY PARA NIVELES DE EXPERINCIA UBNICOS
            t4.categoria_dificultad AS DIFICULTAD,


            FROM {project_id}.{dataset_id1}.{table_id1} as t1                    # registros originales de la tabla maestro de contenido

                  inner join {project_id}.{dataset_id1}.{table_id2} as t2        # se cruzan en interseccion con la tabla maestro de slides
                  ON t1.idslide = t2.idtransparencia_completo

                  inner join {project_id}.{dataset_id2}.{table_id3} as t3        # se cruzan en interseccion con la tabla de slides generadas
                  ON t1.idslide = t3.idslide

                  inner join {project_id}.{dataset_id3}.{table_id4} as t4        # se cruzan en interseccion con la tabla de dificultad de las actividades
                  ON t1.idslide = t4.id_ejercicio

                  WHERE t1.idcursooriginal=0 AND t2.tipo_slide='{par3}' AND t1.nivel_curso_es ='{par2}' AND t1.asignatura_curso_es='{par1}'

                  GROUP BY t4.categoria_dificultad
    '''
    df_dificultad=client.query(dificultad).to_dataframe()
    df_dificultad.replace('Sin definir', ' ', inplace=True)
    return(df_dificultad)



In [ ]:
df_dificultad = dificultad(materias.value,niveles.value,tipos.value)
dificultad=df_dificultad
pd.set_option ("display.max_colwidth",None)
lista_dificultad = dificultad['DIFICULTAD']
dificultad=widgets.Dropdown(options=lista_dificultad,description = 'DIFI')
pd.set_option ("display.max_colwidth",None)
display(dificultad)

Dropdown(description='DIFI', options=('Medio', 'Facil', 'Dificil'), value='Medio')

In [ ]:
  def duracion(par1,par2,par3,par4):
                                                                   ## MODULO 3 : CONTEOS DE LA BASE DE BIGQUERY
                                                                   # ESTA QUEY NOS PERMITIRA SABER LOS REGISTROS QUE TIENE LA BASE ORIGINAL ASI COMO SU CONTENIDO
    dataset_id1 ='maestro'                                         # PARA PODER CONTROLAR LOS CONTENIDOS DE CADA ETIQUETA.
    dataset_id2 ='ai_content'                                      # EN CASO DE NECESITAR MAS CAMPOS DE RESTRICCIONES SE AÑADIRIAN PRIMERO AQUI Y DESPUES A LA QUERY SIGUIENTE
    dataset_id3 ='blinkanalytics'                                  # UNA VEZ YA SEPAMOS EL NÚMERO DE REGISTROS QUE HAN DE SALIR.
    table_id1 = 'maestro_contenido'
    table_id2 = 'maestro_slides'
    table_id3 = 'vm_slides_topics'
    table_id4 = 'slides_difficulty'
    par1=materias.value
    par2=niveles.value
    par3=tipos.value
    par4=dificultad.value




    duracion = f'''
            SELECT                                                               #########################   QUERY PARA DURACIONES UNICAS
                   t4.categoria_duracion AS DURACION

            FROM {project_id}.{dataset_id1}.{table_id1} as t1                    # registros originales de la tabla maestro de contenido

                  inner join {project_id}.{dataset_id1}.{table_id2} as t2        # se cruzan en interseccion con la tabla maestro de slides
                  ON t1.idslide = t2.idtransparencia_completo

                  inner join {project_id}.{dataset_id2}.{table_id3} as t3        # se cruzan en interseccion con la tabla de slides generadas
                  ON t1.idslide = t3.idslide

                  inner join {project_id}.{dataset_id3}.{table_id4} as t4        # se cruzan en interseccion con la tabla de dificultad de las actividades
                  ON t1.idslide = t4.id_ejercicio



                  WHERE t1.idcursooriginal=0 AND t4.categoria_dificultad='{par4}' AND
                        t2.tipo_slide='{par3}' AND t1.nivel_curso_es ='{par2}' AND t1.asignatura_curso_es='{par1}'

                  GROUP BY t4.categoria_duracion

    '''
    df_duracion=client.query(duracion).to_dataframe()
    df_duracion.replace('Sin definir', ' ', inplace=True)

    return(df_duracion)



In [ ]:
df_duracion = duracion(materias.value,niveles.value,tipos.value,dificultad.value)
duracion=df_duracion
pd.set_option ("display.max_colwidth",None)
lista_duracion = duracion['DURACION']
duracion=widgets.Dropdown(options=lista_duracion,description = 'DURA')
pd.set_option ("display.max_colwidth",None)
display(duracion)

Dropdown(description='DURA', options=('Largo', 'Corto', 'Muy largo', 'Medio'), value='Largo')

In [ ]:
def recommendations(id,user,min_sim,num_reg): ## MODULO 5 : SELECCION DEL ONJUNTO DE REGISTROS RELEVANTES PARA LA RECOMENDACION                                                                                                                                                                                                                                                ºdef recommendations(id,curso,asignatura,formato,dificultad,duracion,user,min_sim,num_reg): ## MODULO 5 : SELECCION DEL ONJUNTO DE REGISTROS RELEVANTES PARA LA RECOMENDACION
                                                                                           # Los registros relevantes serán los que cumplan unas condiciones determinadas
                                                                                           # y se extraeran a partir de una funcion, en la que incluiremos las restriciones correspondientes
                                                                                           # por el momento  FORMATO,ASIGNATURA, DIFICULTAD Y DURACIÓN.


  from transformers import AutoTokenizer, AutoModel
  from sklearn.metrics.pairwise import cosine_similarity
  from sentence_transformers import SentenceTransformer,util
  import pandas as pd
  import torch
  import traceback

  # Primero fijamos los parámetros de la dirección de las tablas que vamos a combinar en la consulta.

  dataset_id1 ='maestro'
  dataset_id2 ='ai_content'
  dataset_id3 ='blinkanalytics'
  table_id1 = 'maestro_contenido'
  table_id2 = 'maestro_slides'
  table_id3 = 'vm_slides_topics'
  table_id4 = 'slides_difficulty'



  asignatura=materias.value
  curso=niveles.value
  formato=tipos.value
  dific=dificultad.value
  durac=duracion.value





  # Controlamos que el id de la slide , el numero de usuario , y los topes de similitud y registros están completados.
  # En caso de que alguno este vacio, salimos de la funcion mostrando un mensaje aclaratorio :


  if (not id) or (id==' ') or (not user) or ( user==' ') or (not min_sim) or min_sim==' ' or (not num_reg) or (num_reg==' ') :
      print('Completar campos que no pueden estar vacios : id, user, min_sim, num_reg' )
      return

  ###################################################################################################################################################

   # Construimos el resto de elementos que nos servirán de referencia SQL / Big Query para realizar la consulta.
   # los parámetros que restringirán el conjunto que extraemos en cada caso.


   # Los datos se leeran directamente de la siguiente consulta SQL que selecciona los registros que cumplan a la vez todas las condiciones
   # Si se requirieran nuevas restricciones , este sería el punto desde donde habría que introducirlas
   # nregistros contendrá el intervalo alrededor de idslide que se busque
  try:
      query = f'''                                                                    # query sql para traerse los registros que aparecen en la seleccion de la funcion
              SELECT * FROM
                          (SELECT t1.idslide AS idslide,
                                  t1.nivel_curso_es AS CURSO,
                                  t1.asignatura_curso_es AS ASIGNATURA,
                                  t2.tipo_slide AS FORMATO,
                                  t4.categoria_dificultad AS DIFICULTAD,
                                  t4.categoria_duracion AS DURACION,
                                  concat(t3.topic1,"/",t3.topic2,"/",t3.topic3,"/",t3.topic4,"/",t3.topic5,"/",t3.topic6,"/",t3.topic7,"/",t3.topic8,"/",t3.topic9,"/",t3.topic10) AS topic

                            FROM {project_id}.{dataset_id1}.{table_id1} as t1

                                  inner join {project_id}.{dataset_id1}.{table_id2} as t2
                                  ON t1.idslide = t2.idtransparencia_completo

                                  inner join {project_id}.{dataset_id2}.{table_id3} as t3
                                  ON t1.idslide = t3.idslide


                                  inner join {project_id}.{dataset_id3}.{table_id4} as t4            # PONEMOS LAS CONDICIONES PARA QUEDARNOS CON UNA ACTIVIDAD
                                  ON t1.idslide = t4.id_ejercicio                                    # FUNDAMENTALMENTE VALORES EN LOS CAMPOS CRITERIO CURSO,ASIGNATURA
                                                                                                     # FORMATO , DIFICULTAD Y DURACION .

                                  WHERE   t1.idcursooriginal=0 AND
                                          ((LENGTH('{curso}') > 0 AND t1.nivel_curso_es LIKE '%{curso}%') OR (('{curso}') = ' ' AND t1.nivel_curso_es LIKE '%')) AND
                                          ((LENGTH('{asignatura}') > 0 AND t1.asignatura_curso_es LIKE '%{asignatura}%') OR (('{asignatura}') = ' ' AND t1.asignatura_curso_es LIKE '%')) AND
                                          ((LENGTH('{formato}') > 0 AND t2.tipo_slide LIKE '%{formato}%') OR (('{formato}') = ' ' AND t2.tipo_slide LIKE '%')) AND
                                          ((LENGTH('{dific}') > 0 AND t4.categoria_dificultad LIKE '%{dific}%') OR(('{dific}') = ' ' AND t4.categoria_dificultad LIKE '%')) AND
                                          ((LENGTH('{durac}') > 0 AND t4.categoria_duracion LIKE '%{durac}%') OR (('{durac}') = ' ' AND t4.categoria_duracion LIKE '%'))


                                  GROUP BY t1.idslide,                                                # SI CRITERIO DE CAMPO SE DEJA VACIO TRAE ACTIVIDADES CUALESQUIERA
                                        t1.nivel_curso_es,                                            # QUE SEA EL VALOPR DE ESE CAMPO
                                        t1.asignatura_curso_es,
                                        t2.tipo_slide,
                                        t4.categoria_dificultad,
                                        t4.categoria_duracion,
                                        t3.topic1,
                                        t3.topic2,
                                        t3.topic3,
                                        t3.topic4,
                                        t3.topic5,
                                        t3.topic6,
                                        t3.topic7,
                                        t3.topic8,
                                        t3.topic9,
                                        t3.topic10

                                  ORDER BY  t1.idslide ASC,
                                        t1.nivel_curso_es ASC,
                                        t1.asignatura_curso_es ASC,
                                        t2.tipo_slide ASC,
                                        t4.categoria_dificultad ASC,
                                        t4.categoria_duracion ASC
                              )

                      WHERE idslide not in
                              (SELECT idslide
                                FROM `projectml-345114.trazas_clean.users_actividades`
                                WHERE idusuario={user}
                                )                                                                        # curso , aignatura ,formato, dificultad y duracion
                                                                                      # tambien se trae el topic

    '''
      query2 = f'''                                                                    # query sql para traerse los registros que aparecen en la seleccion de la funcion

                SELECT DISTINCT t1.idslide AS SLIDE,

                       concat(t3.topic1,"/",t3.topic2,"/",t3.topic3,"/",t3.topic4,"/",t3.topic5,"/",t3.topic6,"/",t3.topic7,"/",t3.topic8,"/",t3.topic9,"/",t3.topic10) AS topic

                      FROM {project_id}.{dataset_id1}.{table_id1} as t1

                            inner join {project_id}.{dataset_id1}.{table_id2} as t2
                            ON t1.idslide = t2.idtransparencia_completO

                            inner join {project_id}.{dataset_id2}.{table_id3} as t3
                            ON t1.idslide = t3.idslide


                            inner join {project_id}.{dataset_id3}.{table_id4} as t4       # PONEMOS LAS CONDICIONES PARA QUEDARNOS CON UNA ACTIVIDAD
                            ON t1.idslide = t4.id_ejercicio                               # FUNDAMENTALMENTE VALORES EN LOS CAMPOS CRITERIO CURSO,ASIGNATURA
                                                                                          # FORMATO , DIFICULTAD Y DURACION .

                        WHERE   t1.idslide={id}




                                                                                           # SI CRITERIO DE CAMPO SE DEJA VACIO TRAE ACTIVIDADES CUALESQUIERA


                                                                                           # curso , aignatura ,formato, dificultad y duracion
                                                                                           # tambien se trae el topic

      '''







      DATAFRAME1=client.query(query).to_dataframe()
      fortopic=client.query(query2).to_dataframe()                                      # tenemos sl primer dataframe sobre el que vamos a trabajar


    #########################################################################################################################################################################################
                                                                                    # MODULO 6  : MODULO DE PONDERACION DE RECOMENDACIONES
      verbatim = str(fortopic.loc[fortopic['SLIDE'] == id,'topic'])

      print(verbatim)
                                                                                    # lo extraemos del conjunto de interes inicial para asegurarnos de incluirlo de nuevo tras eliminar

                                                                                   # las actividades ya realizadas




                                                                                    # empezamos extrayendo el topico correspondiente al slide de referencia

                                                                                    # cargamos la versión de los modelos pre-entrenados y el comado 'util' para operar con tranformers
      model = SentenceTransformer('all-mpnet-base-v2')
                                                                                    # elegimos el modelo de trabajo , que teóricamente es el de más precisión
      DATAFRAME2=DATAFRAME1.reset_index(drop=True)
      DATAFRAME3=DATAFRAME2
      entrada_embedding=model.encode([verbatim])
      embeddings=model.encode(DATAFRAME3['topic'])



      similitudes = util.pytorch_cos_sim(entrada_embedding,embeddings)

                                                                                     # Lista ordenada de similitdes del vector input y el universo.
                                                                                     # Nos quedamos solo con los indices y seleccionamos los 10 más parecidos.



                                                                                    # cálculo de las similitudes entre el imput y el universo disponible
      for K in similitudes:
            DATAFRAME3['similitud']=K

      DATAFRAME3['similitud']=DATAFRAME3['similitud'].astype(float)
                                                                                    # creamos campo similitud en funcion de las que hemos calculado
      DATAFRAME4=DATAFRAME3.sort_values(by='similitud', ascending=False)

                                                                                    # ordenamos de forma descendente



    ########################################################################################################################################################################################
                                                                                    # MODULO 7 : PREPARACION DE LA SALIDA

      FINAL=DATAFRAME4[DATAFRAME4['similitud'] > min_sim]                           # Primero nos quedamos con los registros con similitud aceptada
      FINAL=FINAL.head(num_reg)
                                                                                    # Limitamos el numero extraible a la cantidad deseada
      FINAL2=FINAL[FINAL['idslide'] != id]

      largo=len(FINAL2['similitud'])



      if largo>0:

        FINAL2['prioridad'] = pd.cut(FINAL2['similitud'], bins=3, labels=['baja','normal','alta'])
                                                                                                                                # codificamos la salida en funcion de la similitud
        pd.set_option ("display.max_colwidth",None)
        pd.set_option ("display.max_rows", 500)
        json_final=FINAL2.to_json(orient='records')
        return(FINAL2[['idslide','topic','similitud']])                                                                                                                            # y limitamos los registros de salida
        #return(json_final)

      else:
        print("Combinacion no factible1 ; Seleccione otra combinacion")                                                                                                                     # puede ocurrir que el dataframe de salida sea cero y aparecería un error
                                                                                                                                # si no preveemos la excepcion.

  except : print("Combinacion no factible2 ; Seleccione otra combinacion")




In [ ]:
data_sel=recommendations(35744596,123,0.2,10)

0    question/answers/selection/name/pronunciation/spelling/choices/identification/language/learning
Name: topic, dtype: object


In [ ]:
data_sel

,idslide,topic,similitud
6,30551386,reading/comprehension/fill-in/parents/teaching/subjects/village/fishing/weather/andes,0.546979
3,29507295,vocabulary/news/invention/transmit/century/internet/business/coffee/machines/travel,0.508205
11,134349265,dialogues/english/vocabulary/fill-in/comprehension/listening/speaking/grammar/phrases/communication,0.493652
1,29506545,adjectives/description/hair/eyes/length/style/colour/size/type/order,0.431118
7,29616825,questions/fill-in-the-blanks/vocabulary/clothing/colors/sizes/preferences/pricing/fitting/english,0.423524
5,107321405,examples/giving/phrases/rules/text/sentence/start/complete/find/read,0.414150
9,29393545,vocabulary/gravity/float/crumbs/waste/miss/tie/water/birds/space,0.399797
8,135444417,irregular/regular/past/participle/grammar/tenses/complete/fill/in/exercise,0.388142
4,29513125,verbs/activities/party/course/yoga/help/stay/surfing/camping/match,0.380427
0,135353305,play/table/game/board/video/money/cinema/bored/do/shall,0.328245
